In [1]:
import pandas as pd
import numpy as np
import requests
import time

In [ ]:
pip install pandas-gbq

## Connect to BQ


In [3]:
from google.oauth2 import service_account
from google.cloud import bigquery


In [4]:
credentials = service_account.Credentials.from_service_account_file(
'd:/Users/Ramona/Desktop/music-recommendation-system-24-3d0d21fb1f8b.json')
# music-recommendation-system-24-3d0d21fb1f8b.json is the service account JSON file. Save the file locally on your device and add the path here.
# make sure that the slashes in the path are '/' and not '\'

project_id = 'music-recommendation-system-24'
client = bigquery.Client(credentials= credentials,project=project_id)

Query BQ

In [54]:
query_job = client.query("""
   SELECT
      *
   FROM `music-recommendation-system-24.top_100.2024_spotify_top_100_cleaned_csv`
   """)
#put the query in three quote marks

results = query_job.result() # Wait for the job to complete.

rows = [dict(row) for row in results]

# Convert the list of dictionaries to a DataFrame
df_bq = pd.DataFrame(rows)

Getting the Access Token using Client_id and Client_secret

In [ ]:
import requests
import base64

# Replace with your own client ID and client secret
CLIENT_ID = ''
CLIENT_SECRET = ''

def get_access_token(client_id, client_secret):
    # Base64 encode client_id and client_secret
    auth_str = f"{client_id}:{client_secret}"
    b64_auth_str = base64.b64encode(auth_str.encode()).decode()

    # Define the token endpoint and headers
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": f"Basic {b64_auth_str}"
    }
    data = {
        "grant_type": "client_credentials"
    }

    # Make the request to get the token
    response = requests.post(url, headers=headers, data=data)
    if response.status_code == 200:
        token = response.json().get('access_token')
        return token
    else:
        raise Exception("Could not get access token")

In [67]:
# Call the function to get the access token

token = get_access_token(CLIENT_ID, CLIENT_SECRET)

# Attempt to get all artists' info and store them as lists instead of a value

In [68]:
# Create a dictionary to store data. Later on will be converted to a Data Frame

d = {
    'track_id' : [],
    'artist_id' : [],
    'artist_name' : [],
    'artist_followers' : [],
    'artist_popularity' : []
}

# iterate over track_ids

for id in df_bq['track_id']:

    # Add track_id
    d['track_id'].append(id)

    # use the track id to make an API call to get the artist id
    endpoint = f"https://api.spotify.com/v1/tracks/{id}"


    # API call
    response = requests.get(endpoint, headers={'Authorization' : f"Bearer {token}"})
    time.sleep(30)
    # give the response a variable name
    track_response = response.json()

    # Extract artist_id (Here, I4m only taking the first element of the list 'artists', you should iterate over all of them)

    art_names = []
    art_followers = []
    art_ids = []
    art_popularities = []

    # get a list of all artists' ids

    for artist in track_response['artists'] :

      art_ids.append(artist['id'])

    # You should iterate over all elements of the list that contains all artists_ids and make an API call for each of them
    # artist endpoint to get artist's info

    for artist_id in art_ids :

      artist_endpoint = f"https://api.spotify.com/v1/artists/{artist_id}"

      artist_response = requests.get(artist_endpoint, headers={'Authorization' : f"Bearer {token}"}).json()

      # Get artist data

      followers = artist_response['followers']['total']

      name = artist_response['name']

      popularity = artist_response['popularity']

      art_names.append(name)
      art_followers.append(followers)
      art_popularities.append(popularity)


    # append data to the dictionary
    # SHould add the populated lists of info about all artists
    d['artist_name'].append(art_names)
    d['artist_followers'].append(art_followers)
    d['artist_popularity'].append(art_popularities)
    d['artist_id'].append(art_ids)
    



In [69]:
pd.DataFrame(d)

,track_id,artist_id,artist_name,artist_followers,artist_popularity
0,0nrRP2bk19rLc0orkWPQk2,[1vCWHaC5f2uS3yhpwWbIA6],[Avicii],[23107521],[81]
1,7BqHUALzNBTanL6OvsqmC1,"[64KEffDW9EtZ1y2vBYgq8T, 7EQ0qTo7fWT7DPxmxtSYEc]","[Marshmello, Bastille]","[33882258, 5582338]","[84, 76]"
2,2YWjW3wwQIBLNhxWKBQd16,"[738wLrAtLtCtFOLvQBXOXp, 0bdfiayQAKewqEvaU6rXC...","[Major Lazer, MØ, DJ Snake]","[6206097, 1458940, 8910877]","[77, 72, 79]"
3,0lYBSQXN6rCTvUZvg9S0lU,"[540vIaP2JwjQb9dm3aArA4, 1uNFoZAHBGtllmzznpCI3s]","[DJ Snake, Justin Bieber]","[8910877, 78641625]","[79, 91]"
4,6v3KW9xbzN5yKLt9YKDYA2,"[7n2wHs1TKAczGzO7Dd2rGr, 4nDoRrQiYLoBzwC5BhVJzF]","[Shawn Mendes, Camila Cabello]","[43635681, 34041015]","[85, 82]"
...,...,...,...,...,...
95,2Ch7LmS7r2Gy2kc64wv3Bz,[1Xyo4u8uXC1ZmMpatF05PJ],[The Weeknd],[94260543],[96]
96,5KReHDo8W8w9bKTY3KxPy3,[0gadJ2b9A4SKsB1RFkBb66],[Passenger],[3419887],[72]
97,7yq4Qj7cqayVTp3FF9CWbm,[10exVja0key0uqUkk6LJRT],[Vance Joy],[3199643],[78]
98,3hRV0jL3vUpRrcy398teAU,[6ltzsmQQbmdoHHbLZ4ZN25],[Lord Huron],[1765546],[78]


In [70]:
df_artists = pd.DataFrame(d)
df_artists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   track_id           100 non-null    object
 1   artist_id          100 non-null    object
 2   artist_name        100 non-null    object
 3   artist_followers   100 non-null    object
 4   artist_popularity  100 non-null    object
dtypes: object(5)
memory usage: 4.0+ KB


In [74]:
df_artists

,track_id,artist_id,artist_name,artist_followers,artist_popularity
0,0nrRP2bk19rLc0orkWPQk2,[1vCWHaC5f2uS3yhpwWbIA6],[Avicii],[23107521],[81]
1,7BqHUALzNBTanL6OvsqmC1,"[64KEffDW9EtZ1y2vBYgq8T, 7EQ0qTo7fWT7DPxmxtSYEc]","[Marshmello, Bastille]","[33882258, 5582338]","[84, 76]"
2,2YWjW3wwQIBLNhxWKBQd16,"[738wLrAtLtCtFOLvQBXOXp, 0bdfiayQAKewqEvaU6rXC...","[Major Lazer, MØ, DJ Snake]","[6206097, 1458940, 8910877]","[77, 72, 79]"
3,0lYBSQXN6rCTvUZvg9S0lU,"[540vIaP2JwjQb9dm3aArA4, 1uNFoZAHBGtllmzznpCI3s]","[DJ Snake, Justin Bieber]","[8910877, 78641625]","[79, 91]"
4,6v3KW9xbzN5yKLt9YKDYA2,"[7n2wHs1TKAczGzO7Dd2rGr, 4nDoRrQiYLoBzwC5BhVJzF]","[Shawn Mendes, Camila Cabello]","[43635681, 34041015]","[85, 82]"
...,...,...,...,...,...
95,2Ch7LmS7r2Gy2kc64wv3Bz,[1Xyo4u8uXC1ZmMpatF05PJ],[The Weeknd],[94260543],[96]
96,5KReHDo8W8w9bKTY3KxPy3,[0gadJ2b9A4SKsB1RFkBb66],[Passenger],[3419887],[72]
97,7yq4Qj7cqayVTp3FF9CWbm,[10exVja0key0uqUkk6LJRT],[Vance Joy],[3199643],[78]
98,3hRV0jL3vUpRrcy398teAU,[6ltzsmQQbmdoHHbLZ4ZN25],[Lord Huron],[1765546],[78]


In [ ]:
#merge df_bq with df_artists
df_bq = pd.merge(df_bq, df_artists, on='track_id')

In [ ]:
#save merged to gbq
# Define your project ID and BigQuery table ID (including dataset name)
project_id = 'music-recommendation-system-24'
table_id = 'top_100.df_artists'  # Replace 'your_dataset_name' with your actual dataset name

# Load the credentials from your service account file
credentials = service_account.Credentials.from_service_account_file(
    'd:/Users/Ramona/Desktop/music-recommendation-system-24-3d0d21fb1f8b.json'
)

# Save DataFrame to BigQuery
df_bq.to_gbq(
    destination_table=table_id,
    project_id=project_id,
    credentials=credentials,
    if_exists='replace'  # Use 'append' to add data if the table already exists
)

print("DataFrame saved to BigQuery table 'df_artists'.")

C:\Users\ramon\AppData\Local\Temp\ipykernel_48332\3975331509.py:12: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df_artists.to_gbq(
100%|██████████| 1/1 [00:00<?, ?it/s]

DataFrame saved to BigQuery table 'df_artists'.
